In [19]:
!git clone https://github.com/TheDunkinArt/face_detect_python.git

Cloning into 'face_detect_python'...
Updating files:  11% (1321/11796)
Updating files:  12% (1416/11796)
Updating files:  13% (1534/11796)
Updating files:  14% (1652/11796)
Updating files:  15% (1770/11796)
Updating files:  16% (1888/11796)
Updating files:  17% (2006/11796)
Updating files:  18% (2124/11796)
Updating files:  19% (2242/11796)
Updating files:  20% (2360/11796)
Updating files:  21% (2478/11796)
Updating files:  22% (2596/11796)
Updating files:  22% (2637/11796)
Updating files:  23% (2714/11796)
Updating files:  24% (2832/11796)
Updating files:  25% (2949/11796)
Updating files:  26% (3067/11796)
Updating files:  27% (3185/11796)
Updating files:  28% (3303/11796)
Updating files:  29% (3421/11796)
Updating files:  30% (3539/11796)
Updating files:  31% (3657/11796)
Updating files:  31% (3751/11796)
Updating files:  32% (3775/11796)
Updating files:  33% (3893/11796)
Updating files:  34% (4011/11796)
Updating files:  35% (4129/11796)
Updating files:  36% (4247/11796)
Updating fi

In [20]:
pip install pickleshare

Note: you may need to restart the kernel to use updated packages.


In [21]:
cd face_detect_python

C:\Users\Administrator\Desktop\Saadi1.0\face_detect_python\face_detect_python


In [22]:
ls

 Volume in drive C is Windows Server
 Volume Serial Number is F6DB-3FC4

 Directory of C:\Users\Administrator\Desktop\Saadi1.0\face_detect_python\face_detect_python

09/04/2024  02:25 AM    <DIR>          .
09/04/2024  02:24 AM    <DIR>          ..
09/04/2024  02:25 AM    <DIR>          Dataset
09/04/2024  02:25 AM         4,377,486 Face_Mask_Detection.ipynb
09/04/2024  02:25 AM           561,335 haarcascade_frontalface_alt2.xml
09/04/2024  02:25 AM        11,488,344 mask_recog.h5
09/04/2024  02:25 AM             4,957 ReadMe-Important.txt
               4 File(s)     16,432,122 bytes
               3 Dir(s)  177,646,174,208 bytes free


In [23]:
!pip install --upgrade tensorflow

In [24]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [25]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [26]:
def build_model():
    baseModel = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    headModel = baseModel.output
    headModel = GlobalAveragePooling2D()(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(3, activation="softmax")(headModel)  # Change this to match the number of classes

    model = Model(inputs=baseModel.input, outputs=headModel)
    return model

In [27]:
def load_data(root_dir):
    data_gen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        validation_split=0.2
    )

    train_gen = data_gen.flow_from_directory(
        root_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse',  # Ensure this matches with your labels
        subset='training'
    )

    val_gen = data_gen.flow_from_directory(
        root_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse',  # Ensure this matches with your labels
        subset='validation'
    )

    return train_gen, val_gen

In [28]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import matplotlib
print("Matplotlib version:", matplotlib.__version__)


Matplotlib version: 3.9.2


In [2]:
from PIL import Image
print("Pillow is installed correctly.")


Pillow is installed correctly.


In [14]:
pip install --user opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
print(cv2.getBuildInformation())


General configuration for OpenCV 4.10.0 =====================================
  Version control:               4.10.0

  Platform:
    Timestamp:                   2024-06-17T17:58:36Z
    Host:                        Windows 10.0.17763 AMD64
    CMake:                       3.24.2
    CMake generator:             Visual Studio 14 2015
    CMake build tool:            MSBuild.exe
    MSVC:                        1900
    Configuration:               Debug Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (16 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX
      AVX (8 files):             + SSSE3 SSE4_1 POPCNT SSE4_2 AVX
      AVX2 (36 files):           + SSSE3 SSE4_

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

print("TensorFlow version:", tf.__version__)

# Conditional import for cv2_imshow based on environment
try:
    from google.colab.patches import cv2_imshow  # Import cv2_imshow for displaying images in Colab
    using_colab = True
except ImportError:
    using_colab = False
    cv2_imshow = None  # Set to None if cv2_imshow is not available

def display_image(image, title="Image"):
    if using_colab and cv2_imshow:
        cv2_imshow(image)
    else:
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis('off')  # Hide axes
        plt.show()

# Step 6: Build the model
def build_model(num_classes):
    baseModel = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    headModel = baseModel.output
    headModel = GlobalAveragePooling2D()(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(num_classes, activation="sigmoid")(headModel)  # Use sigmoid for binary classification

    model = Model(inputs=baseModel.input, outputs=headModel)
    return model

# Step 7: Load data and rescale images
def load_data(root_dir):
    data_gen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        validation_split=0.2
    )

    train_gen = data_gen.flow_from_directory(
        root_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',  # Use binary for binary classification
        subset='training'
    )

    val_gen = data_gen.flow_from_directory(
        root_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',  # Use binary for binary classification
        subset='validation'
    )

    return train_gen, val_gen

# Step 8: Compile the model with chosen parameters
def compile_model(model):
    learning_rate = 1e-4
    opt = Adam(learning_rate=learning_rate)
    model.compile(
        loss="binary_crossentropy",  # Use binary crossentropy for binary classification
        optimizer=opt,
        metrics=["accuracy"]
    )

# Step 9: Train the model
def train_model(model, train_gen, val_gen, model_path="best_model.keras"):
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True)

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=20,
        callbacks=[early_stopping, checkpoint]
    )

    return history

# Step 10: Evaluate the model
def evaluate_model(model, val_gen):
    val_loss, val_acc = model.evaluate(val_gen)
    print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")

# Step 11: Make predictions on the test set
def test_model(model, test_dir):
    test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'  # Use binary for binary classification
    )

    predictions = model.predict(test_gen)
    return predictions, test_gen

# Step 12: Face mask detection on sample images
def face_mask_detector(frame, model):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt2.xml")
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)

    if len(faces) == 0:
        print("No faces detected in the image.")
        return None

    for (x, y, w, h) in faces:
        face_frame = frame[y:y+h, x:x+w]
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (224, 224))
        face_frame = img_to_array(face_frame)
        face_frame = np.expand_dims(face_frame, axis=0)
        face_frame = preprocess_input(face_frame)
        preds = model.predict(face_frame)

        mask_prob = preds[0][0]  # Single probability value if using sigmoid
        label = "Mask" if mask_prob > 0.5 else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        label = "{}: {:.2f}%".format(label, mask_prob * 100)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 3)

    return frame

# Step 13: Main Execution
def main():
    root_dir = 'Dataset/Train'  # Updated root directory path
    test_dir = 'Dataset/Test/'  # Test directory for predictions
    model_path = "best_model.keras"

    if not os.path.isdir(root_dir):
        print(f"Dataset directory not found: {root_dir}")
        return

    # Load data
    train_gen, val_gen = load_data(root_dir)
    num_classes = 1  # Binary classification
    print(f"Loaded {train_gen.samples} training images and {val_gen.samples} validation images.")
    print("Class indices:", train_gen.class_indices)
    print("Unique labels in training data:", np.unique(train_gen.classes))
    print("Unique labels in validation data:", np.unique(val_gen.classes))

    # Check if a pre-trained model exists
    if os.path.exists(model_path):
        print(f"Loading pre-trained model from {model_path}...")
        model = tf.keras.models.load_model(model_path)
    else:
        # Build and compile model
        model = build_model(num_classes)
        compile_model(model)

        # Train model
        train_model(model, train_gen, val_gen, model_path)

    # Evaluate model
    evaluate_model(model, val_gen)

    # Test model on the test set
    predictions, test_gen = test_model(model, test_dir)
    print(f"Predictions on test data: {predictions}")
    print("Test labels:", test_gen.classes)

    # Test detection on some images (single image detection)
    sample_image_path = os.path.join(test_dir, 'WithMask/sample_image.png')  # Example image path
    input_image = cv2.imread(sample_image_path)

    if input_image is None:
        print(f"Failed to load image for detection: {sample_image_path}")
    else:
        output = face_mask_detector(input_image, model)
        if output is None:
            print("No faces were detected in the image.")
        else:
            display_image(output, title="Face Mask Detection Output")

            # If not using Colab, add OpenCV display functionality
            if not using_colab and cv2_imshow:
                cv2.waitKey(0)
                cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


TensorFlow version: 2.17.0
Dataset directory not found: Dataset/Train
